## Estudio del tráfico en el área urbana de Madrid

In [1]:
import pandas as pd
import math
from datetime import datetime

### Carga de los datos descargados de la página de datos abiertos del Ayuntamiento de Madrid

Los datos se actualizan de manera mensual. Vienen en archivos _.csv_ de entre 600MB y 800MB aproximadamente.

In [3]:
#Importar CSV de los datos de tráfico de un mes. En este caso, Nov19. 
##Peso:667.8MB##
trf1911 = pd.read_csv("11-2019.csv", sep = ";", header = "infer", encoding='iso-8859-1')

trf1911.head()

,id,fecha,tipo_elem,intensidad,ocupacion,carga,vmed,error,periodo_integracion
0,1001,2019-11-01 00:00:00,M30,1320,3.0,0,50.0,N,5
1,1001,2019-11-01 00:15:00,M30,1260,3.0,0,54.0,N,5
2,1001,2019-11-01 00:30:00,M30,948,3.0,0,56.0,N,5
3,1001,2019-11-01 00:45:00,M30,888,3.0,0,59.0,N,5
4,1001,2019-11-01 01:00:00,M30,972,3.0,0,59.0,N,5


In [22]:
#Dimensiones: 11.2M filas, 9 columnas
trf1911.shape

(11186837, 9)

### Selección de datos últiles

Se elimina aquello superfluo ya que: (i) será más fácil de tratar y (ii) funcionará más rápido al ser más ligero

Se seleccionan las siguientes columnas:
- **id:** indenticador único de la estación de medida de tráfico (existen cerca de 4.000 actualmente)
- **fecha:** momento en de la medida, en intervalos de 15'
- **tipo_elem:** si se trata de área urbana (URB) o suburbana (M30)
- **intensidad:** vehículos/hora. Valores negativos implica ausencia de datos

In [4]:
#Seleccionar las columnas que son de utilidad para el estudio
trf1911_filtrado = trf1911[["id", "fecha", "tipo_elem",'intensidad']]
trf1911_filtrado.columns = ["station_id", "fecha", "magnitud_id",'value']

#Filtrar URB, el área de estudio
trf1911_filtrado = trf1911_filtrado[(trf1911_filtrado["magnitud_id"] == "URB")]

#Asignar una magnitud_id numérica 
#para que el objeto sea del tipo que requiere el evento del esquema estrella de DB
#**Este paso se realiza al pertenecer a un proyecto más grande. No requerido en otro contexto**#
trf1911_filtrado.magnitud_id = trf1911_filtrado.magnitud_id.replace({"URB": 52})

trf1911_filtrado.head()

,station_id,fecha,magnitud_id,value
133738,3395,2019-11-01 00:00:00,52,216
133739,3395,2019-11-01 00:15:00,52,228
133740,3395,2019-11-01 00:30:00,52,227
133741,3395,2019-11-01 00:45:00,52,248
133742,3395,2019-11-01 01:00:00,52,220


In [5]:
#Dimensiones: 10M filas, 4 columnas
trf1911_filtrado.shape

(10007056, 4)

### Dividir la columna _fecha_ por día y hora para poder trabajar con ello

In [6]:
#Pasar de objeto a formato fecha
trf1911_filtrado['fecha'] = pd.to_datetime(trf1911_filtrado['fecha'])

#Añadir columnas separando la fecha y hora para poder trabajar con ello
trf1911_filtrado['day_id'] = trf1911_filtrado['fecha'].dt.date
trf1911_filtrado['time_id'] = trf1911_filtrado['fecha'].dt.hour

trf1911_filtrado.head()

,station_id,fecha,magnitud_id,value,day_id,time_id
133738,3395,2019-11-01 00:00:00,52,216,2019-11-01,0
133739,3395,2019-11-01 00:15:00,52,228,2019-11-01,0
133740,3395,2019-11-01 00:30:00,52,227,2019-11-01,0
133741,3395,2019-11-01 00:45:00,52,248,2019-11-01,0
133742,3395,2019-11-01 01:00:00,52,220,2019-11-01,1


In [7]:
##Quitar los guiones de la fecha para adaptarlo a la DB##
#**Este paso se realiza al pertenecer a un proyecto más grande. No requerido en otro contexto**#
#Es lento en ejecución#

trf1911_filtrado = trf1911_filtrado.astype({'day_id': 'str'})
trf1911_filtrado['day_id'] = trf1911_filtrado.apply(lambda row: row.day_id.replace('-', ''), axis=1)
trf1911_filtrado.head()

,station_id,fecha,magnitud_id,value,day_id,time_id
133738,3395,2019-11-01 00:00:00,52,216,20191101,0
133739,3395,2019-11-01 00:15:00,52,228,20191101,0
133740,3395,2019-11-01 00:30:00,52,227,20191101,0
133741,3395,2019-11-01 00:45:00,52,248,20191101,0
133742,3395,2019-11-01 01:00:00,52,220,20191101,1


In [8]:
#Dimensiones: 10M filas, 6 columnas
trf1911_filtrado.shape

(10007056, 6)

### Agrupar los datos por horas de cara a un futuro análisis

In [9]:
# Identificar las filas con intensidad negativa (lo cual quiere decir sin datos)
indexNames = trf1911_filtrado[ trf1911_filtrado['value'] < 0 ].index
 
# Eliminar esas filas del DataFrame
trf1911_filtrado.drop(indexNames , inplace=True)

trf1911_filtrado.shape

(10007056, 6)

In [10]:
#Agrupar por horas sumando las intensidades
trf1911_filtrado_horas = trf1911_filtrado.groupby(['station_id','day_id','time_id','magnitud_id'])["value"].sum().reset_index()

#Ordenar columnas para que coincida con el orden de la DB
#**Este paso se realiza al pertenecer a un proyecto más grande. No requerido en otro contexto**#
trf1911_filtrado_horas = trf1911_filtrado_horas[['station_id','day_id','time_id','magnitud_id','value']]

trf1911_filtrado_horas.head()

,station_id,day_id,time_id,magnitud_id,value
0,3395,20191101,0,52,919
1,3395,20191101,1,52,757
2,3395,20191101,2,52,651
3,3395,20191101,3,52,493
4,3395,20191101,4,52,518


In [12]:
#Dimensiones: 2.5M filas, 5 columnas
trf1911_filtrado_horas.shape

(2536870, 5)

### Descargar csv con el formato actual

In [13]:
trf1911_filtrado_horas.to_csv(r'trf1911_filtrado.csv', index=False, sep = ";", encoding='utf-8')

###Peso: 60.9MB###

### Trabajar con varios documentos de tráfico de Madrid

De cara a realizar una comparativa en un eje temporal determinado se puede definir un método con los pasos propuestos o ejectuar los mismos para cada uno de los archivos y concaternarlos haciendo uso de `.append`.
De nuevo se realizaría `.to_csv` para evitar correr el código descrito en este documento en más de una ocasión dados los tiempos de ejecución

## Conclusión
Ejecutando este código se consigue obtener un _.csv_ de un tamaño 10 veces menor que el original. 
Con ello se consigue un uso más fluido de los datos facilitando el análisis de los mismos